In [1]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_csv('/Volumes/SANDISK/Video CV/traffic_lights/Annotations/Annotations/daySequence1/frameAnnotationsBOX.csv', sep=';')
df.head()

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin file,Origin frame number,Origin track,Origin track frame number
0,dayTest/daySequence1--00000.jpg,stop,706,478,718,500,dayTest/daySequence1/Day1EveningShutter0.00020...,0,dayTest/daySequence1/Day1EveningShutter0.00020...,0
1,dayTest/daySequence1--00001.jpg,stop,705,475,720,497,dayTest/daySequence1/Day1EveningShutter0.00020...,1,dayTest/daySequence1/Day1EveningShutter0.00020...,1
2,dayTest/daySequence1--00002.jpg,stop,707,476,719,494,dayTest/daySequence1/Day1EveningShutter0.00020...,2,dayTest/daySequence1/Day1EveningShutter0.00020...,2
3,dayTest/daySequence1--00005.jpg,stop,708,474,720,492,dayTest/daySequence1/Day1EveningShutter0.00020...,5,dayTest/daySequence1/Day1EveningShutter0.00020...,5
4,dayTest/daySequence1--00006.jpg,stop,707,470,722,492,dayTest/daySequence1/Day1EveningShutter0.00020...,6,dayTest/daySequence1/Day1EveningShutter0.00020...,6


In [61]:
df_copy = df.copy()
df_copy = df_copy.drop(columns=['Origin frame number', 'Origin track', 'Origin track frame number', 'Origin file'])
df_copy = df_copy.rename(columns={'Annotation tag': 'class', 'Upper left corner X': 'xMin', 'Upper left corner Y': 'yMin', 'Lower right corner X': 'xMax', 'Lower right corner Y': 'yMax'})
df_copy = df_copy.drop(df_copy.index[df_copy['class'] == 'warningLeft'])
df_copy = df_copy.drop(df_copy.index[df_copy['class'] == 'stopLeft'])

df_copy.head()

,Filename,class,xMin,yMin,xMax,yMax
0,dayTest/daySequence1--00000.jpg,stop,706,478,718,500
1,dayTest/daySequence1--00001.jpg,stop,705,475,720,497
2,dayTest/daySequence1--00002.jpg,stop,707,476,719,494
3,dayTest/daySequence1--00005.jpg,stop,708,474,720,492
4,dayTest/daySequence1--00006.jpg,stop,707,470,722,492


In [62]:
width = 1280
height = 960

In [63]:
classList = []

xMin = df_copy['xMin']
xMax = df_copy['xMax']
yMin = df_copy['yMin']
yMax = df_copy['yMax']

xCen = (xMin + xMax) / 2 / width
yCen = (yMin + yMax) / 2 / height

w = (xMax - xMin) / width
h = (yMax - yMin) / height

boxName = df_copy['class']


In [64]:
frame = {'Class': boxName, 'X': xCen, 'Y': yCen, 'w': w, 'h': h}

YOLOdf = pd.DataFrame(frame)
YOLOdf.head()

,Class,X,Y,w,h
0,stop,0.556250,0.509375,0.009375,0.022917
1,stop,0.556641,0.506250,0.011719,0.022917
2,stop,0.557031,0.505208,0.009375,0.018750
3,stop,0.557813,0.503125,0.009375,0.018750
4,stop,0.558203,0.501042,0.011719,0.022917


In [65]:
YOLOdf['Class'] = YOLOdf['Class'].map({'stop': '0', 'go': '1', 'warning': '2'})
YOLOdf

,Class,X,Y,w,h
0,0,0.556250,0.509375,0.009375,0.022917
1,0,0.556641,0.506250,0.011719,0.022917
2,0,0.557031,0.505208,0.009375,0.018750
3,0,0.557813,0.503125,0.009375,0.018750
4,0,0.558203,0.501042,0.011719,0.022917
...,...,...,...,...,...
7895,1,0.748437,0.214062,0.056250,0.130208
7896,1,0.802344,0.191146,0.060937,0.140625
7897,1,0.859375,0.163542,0.065625,0.145833
7898,1,0.922266,0.135417,0.082031,0.177083


In [84]:
import re

files_list = list(df_copy['Filename'])

In [87]:
files_names = []

for file in files_list:
    name = re.search(('/(.*)j'), file).group(1)
    files_names.append(name)

In [88]:
YOLOdf['Class'].iloc[0]

'0'

In [93]:
import os

for i in range(len(files_names)):
    with open('train/' + files_names[i] + 'txt', 'w') as f:
        class_name, x, y, w, h = YOLOdf['Class'].iloc[i], YOLOdf['X'].iloc[i], YOLOdf['Y'].iloc[i], YOLOdf['w'].iloc[i], YOLOdf['h'].iloc[i]
        f.write("%s %.6f %.6f %.6f %.6f\n" % (class_name, x, y, w, h))
